In [ ]:
%load_ext autoreload

%autoreload 2

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Test Transforms

In [ ]:
import os

import lightning as L
import omegaconf
import pyrootutils
import torch
import torchshow as ts
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger, WandbLogger
from torchvision.utils import make_grid

from gaussian_denoiser import data, transforms

In [ ]:
root = pyrootutils.setup_root(
    search_from=".",
    indicator="pyproject.toml",
    project_root_env_var=True,
    dotenv=True,
    pythonpath=False,
    cwd=True,
)

PROJECT_ROOT = os.getenv("PROJECT_ROOT")

In [ ]:
cfg = omegaconf.OmegaConf.load(f"{PROJECT_ROOT}/config/train.yaml")

In [ ]:
L.seed_everything(123, workers=True)

## Prepare Data

In [ ]:
import torchvision
from torchvision.transforms import functional as TF

ds = data.InferenceImageDataset(
    image_dir=cfg.data.train_path,
    transform=torchvision.transforms.Compose([torchvision.transforms.CenterCrop((50, 50))]),
)

In [ ]:
image = ds[0]
type(image)
image

## Test AWGN

In [ ]:
tr = transforms.AWGNOnlyTransform(
    min_variance=cfg.data.train_noise_level_interval[0],
    max_variance=cfg.data.train_noise_level_interval[1],
)

In [ ]:
image_noise, noise, var = tr(image)
print(var)

In [ ]:
TF.to_tensor(image).max()
TF.to_tensor(image).min()

In [ ]:
TF.to_tensor(image_noise).max()
TF.to_tensor(image_noise).min()

In [ ]:
(noise / 255.0).max()
(noise / 255.0).min()

In [ ]:
diff = TF.to_tensor(image) - (TF.to_tensor(image_noise) - (noise / 255.0))

torch.testing.assert_close(diff, torch.zeros_like(diff))

## Up Downsampling

In [ ]:
tr = transforms.DownUpsampleTransform(factors=cfg.data.up_down_factors)

In [ ]:
image_noise, noise = tr(image)

In [ ]:
TF.to_tensor(image_noise).max()
TF.to_tensor(image_noise).min()

In [ ]:
type(image)
type(image_noise)

In [ ]:
ts.show(image)

In [ ]:
ts.show(image_noise)

In [ ]:
diff = TF.to_tensor(image) - (TF.to_tensor(image_noise) - (noise / 255.0))

torch.testing.assert_close(diff, torch.zeros_like(diff))

## Test JPEG

In [ ]:
tr = transforms.JPEGTransform(
    min_quality=cfg.data.jpeg_min_max[0], max_quality=cfg.data.jpeg_min_max[1]
)

In [ ]:
image_noise, noise = tr(image)

In [ ]:
ts.show(image)
ts.show(image_noise)

In [ ]:
diff = TF.to_tensor(image) - (TF.to_tensor(image_noise) - (noise / 255.0))

torch.testing.assert_close(diff, torch.zeros_like(diff))

# Combined Transforms

In [ ]:
tr = transforms.CombinedTransform(
    min_quality=cfg.data.jpeg_min_max[0],
    max_quality=cfg.data.jpeg_min_max[1],
    factors=cfg.data.up_down_factors,
    min_variance=cfg.data.train_noise_level_interval[0],
    max_variance=cfg.data.train_noise_level_interval[1],
)

In [ ]:
for i in range(10):
    image_noise, noise = tr(image)
    ts.show(image_noise)

In [ ]:
type(image)
image
img = TF.to_tensor(image)
img.max()

img_rec = TF.to_pil_image(img)
img_rec

torch.testing.assert_close(TF.to_tensor(img_rec), img)

## Test Pipeline

In [ ]:
train_noise = transforms.AWGNOnlyTransform(
    min_variance=cfg.data.train_noise_level_interval[0],
    max_variance=cfg.data.train_noise_level_interval[1],
)
val_noise = transforms.AWGNOnlyTransform(
    min_variance=cfg.data.val_noise_level_interval[0],
    max_variance=cfg.data.val_noise_level_interval[1],
)

data_module = data.DenoisingDataModule(
    train_path=cfg.data.train_path,
    val_path=cfg.data.val_path,
    test_path=cfg.data.test_path,
    batch_size=16,
    patch_size=cfg.data.patch_size,
    train_noise=train_noise,
    val_noise=val_noise,
)

data_module.setup("fit")

In [ ]:
ds_train = data_module.train_dataloader()
clean_batch, noisy_batch, noise_batch = next(iter(ds_train))

In [ ]:
torch.min(noisy_batch)
torch.max(noisy_batch)

torch.min(clean_batch)
torch.max(clean_batch)

torch.min(noise_batch)
noise_batch.view(noise_batch.shape[0], -1).max(0)